# 03 — Portfolio Optimisation

Markowitz Mean-Variance Optimisation (MVO) using `scipy.optimize.minimize` (SLSQP).

- Max Sharpe, Min Variance, and Equal Weight portfolios
- 50-point Efficient Frontier
- Weight breakdown pie charts
- Sensitivity: Sharpe vs constraints
- 10-year backtest: Max Sharpe vs Equal Weight vs 100% Equity (using walk-forward rebalancing)
- 5,000-path Monte Carlo block-bootstrap wealth paths

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from scipy.optimize import minimize
from pathlib import Path

plt.rcParams['figure.dpi'] = 120

DB_PATH = Path('../funds.db')
RISK_FREE_RATE = 0.065  # annualised, nominal

ASSETS = {
    'equity':    {'scheme_code': 120716, 'label': 'Passive Equity (Nifty 50)'},
    'gold':      {'scheme_code': 111954, 'label': 'Gold ETF'},
    'gilt':      {'scheme_code': 119116, 'label': 'Government Bonds'},
    'corp_bond': {'scheme_code': 118987, 'label': 'Corporate Bonds'},
    'short_dur': {'scheme_code': 118780, 'label': 'Short Duration Debt'},
    'liquid':    {'scheme_code': 119568, 'label': 'Liquid (Cash Proxy)'},
}

conn = sqlite3.connect(DB_PATH)
nav_frames = {}
for asset_id, meta in ASSETS.items():
    df = pd.read_sql_query(
        'SELECT date, nav FROM nav WHERE scheme_code = ? AND date >= ? AND date <= ? ORDER BY date',
        conn, params=(meta['scheme_code'], '2013-01-01', '2025-12-31'),
        parse_dates=['date'], index_col='date',
    )
    nav_frames[asset_id] = df['nav'].rename(asset_id)
conn.close()

nav_df = pd.DataFrame(nav_frames).ffill()
monthly_nav = nav_df.resample('ME').last().dropna()
monthly_ret = monthly_nav.pct_change().dropna()

mu    = monthly_ret.mean().values * 12           # annualised expected return
sigma = monthly_ret.cov().values  * 12           # annualised covariance
n     = len(mu)
labels = list(monthly_ret.columns)
label_names = [ASSETS[l]['label'] for l in labels]
print(f'Assets: {labels}\nMonthly return obs: {len(monthly_ret)}')

In [ ]:
# ── Optimisation helpers ─────────────────────────────────────────────────

def portfolio_ret(w):  return float(w @ mu)
def portfolio_vol(w):  return float(np.sqrt(w @ sigma @ w))
def portfolio_sharpe(w):
    v = portfolio_vol(w)
    return (portfolio_ret(w) - RISK_FREE_RATE) / v if v > 1e-10 else 0.0

constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
bounds      = [(0, 1)] * n
w0          = np.ones(n) / n

# Max Sharpe
res_ms = minimize(lambda w: -portfolio_sharpe(w), w0, method='SLSQP',
                  bounds=bounds, constraints=constraints,
                  options={'maxiter': 1000, 'ftol': 1e-12})
w_ms = res_ms.x

# Min Variance
res_mv = minimize(lambda w: portfolio_vol(w), w0, method='SLSQP',
                  bounds=bounds, constraints=constraints,
                  options={'maxiter': 1000, 'ftol': 1e-12})
w_mv = res_mv.x

# Equal Weight
w_ew = np.ones(n) / n

for name, w in [('Max Sharpe', w_ms), ('Min Variance', w_mv), ('Equal Weight', w_ew)]:
    print(f'{name:15s}  ret={portfolio_ret(w)*100:.2f}%  vol={portfolio_vol(w)*100:.2f}%  sharpe={portfolio_sharpe(w):.3f}')

In [ ]:
# ── Efficient Frontier (50 points) ───────────────────────────────────────
ret_min  = portfolio_ret(w_mv)
ret_max  = portfolio_ret(w_ms) * 1.05
targets  = np.linspace(ret_min, ret_max, 50)

frontier = []
for target in targets:
    con = constraints + [{'type': 'eq', 'fun': lambda w, t=target: portfolio_ret(w) - t}]
    res = minimize(lambda w: portfolio_vol(w), w0, method='SLSQP',
                   bounds=bounds, constraints=con, options={'maxiter': 500})
    if res.success:
        frontier.append({'ret': portfolio_ret(res.x)*100, 'vol': portfolio_vol(res.x)*100})

# Random portfolios cloud (500)
rng = np.random.default_rng(42)
random_pts = []
for _ in range(500):
    w = rng.dirichlet(np.ones(n))
    random_pts.append({'ret': portfolio_ret(w)*100, 'vol': portfolio_vol(w)*100,
                       'sharpe': portfolio_sharpe(w)})

fr_vols = [p['vol'] for p in frontier]
fr_rets = [p['ret'] for p in frontier]

fig, ax = plt.subplots(figsize=(10, 6))
rvols = [p['vol'] for p in random_pts]
rrets = [p['ret'] for p in random_pts]
rshrp = [p['sharpe'] for p in random_pts]
sc = ax.scatter(rvols, rrets, c=rshrp, cmap='viridis', alpha=0.45, s=12, zorder=2)
plt.colorbar(sc, ax=ax, label='Sharpe Ratio')

ax.plot(fr_vols, fr_rets, 'b-', linewidth=2.5, label='Efficient Frontier', zorder=3)

named = [
    (w_ms, 'Max Sharpe',   '#3B82F6', 'D', 150),
    (w_mv, 'Min Variance', '#10B981', '^', 150),
    (w_ew, 'Equal Weight', '#F59E0B', 'o', 150),
]
for w, name, color, marker, size in named:
    ax.scatter(portfolio_vol(w)*100, portfolio_ret(w)*100,
               color=color, s=size, marker=marker, zorder=5,
               edgecolors='white', linewidths=1.5, label=name)

ax.set_xlabel('Annualised Volatility (%)')
ax.set_ylabel('Annualised Return (%)')
ax.set_title('Efficient Frontier — Markowitz MVO (2013–2025)', fontsize=13, fontweight='bold')
ax.legend(fontsize=9)
fig.tight_layout()
plt.show()

In [ ]:
# ── Portfolio weight pie charts ───────────────────────────────────────────
COLORS = ['#3B82F6','#F59E0B','#10B981','#6366F1','#EC4899','#8B5CF6']

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
for ax, (w, title) in zip(axes, [(w_ms,'Max Sharpe'), (w_mv,'Min Variance'), (w_ew,'Equal Weight')]):
    nonzero = [(wt, label_names[i]) for i, wt in enumerate(w) if wt > 0.005]
    sizes   = [x[0] for x in nonzero]
    lbls    = [f"{x[1]}\n{x[0]*100:.1f}%" for x in nonzero]
    ax.pie(sizes, labels=lbls, colors=COLORS[:len(sizes)],
           startangle=90, wedgeprops={'edgecolor': 'white', 'linewidth': 1.5})
    ax.set_title(f'{title}\nRet {portfolio_ret(w)*100:.1f}%  Vol {portfolio_vol(w)*100:.1f}%  Sharpe {portfolio_sharpe(w):.2f}',
                 fontsize=10)

fig.suptitle('Portfolio Allocations', fontsize=13, fontweight='bold')
fig.tight_layout()
plt.show()

In [ ]:
# ── Backtest: 2013–2025, annual rebalancing ───────────────────────────────
# Walk-forward: estimate weights on rolling 3-year window, then hold 1 year

def rebalance_weights(ret_history):
    mu_h  = ret_history.mean().values * 12
    sig_h = ret_history.cov().values  * 12
    def neg_sharpe(w):
        vol = np.sqrt(w @ sig_h @ w)
        return -(w @ mu_h - RISK_FREE_RATE) / vol if vol > 1e-10 else 0.0
    res = minimize(neg_sharpe, np.ones(n)/n, method='SLSQP',
                   bounds=[(0,1)]*n, constraints=[{'type':'eq','fun':lambda w:np.sum(w)-1}])
    return res.x if res.success else np.ones(n)/n

months = monthly_ret.index.to_period('M')
start_year = 2016  # need 3 years of history first

bt_ms, bt_ew, bt_eq = {'date':[], 'value':[]}, {'date':[], 'value':[]}, {'date':[], 'value':[]}
val_ms = val_ew = val_eq = 1.0
w_cur = np.ones(n) / n  # initial equal weight

current_year = None
for i, (date, row) in enumerate(monthly_ret.iterrows()):
    yr = date.year
    if yr < start_year:
        continue
    # Rebalance at start of each year
    if yr != current_year:
        current_year = yr
        history = monthly_ret.iloc[max(0, i-36):i]
        if len(history) >= 12:
            w_cur = rebalance_weights(history)

    r = row.values
    val_ms *= (1 + float(w_cur @ r))
    val_ew *= (1 + float(np.ones(n)/n @ r))
    val_eq *= (1 + float(r[labels.index('equity')]))

    for d, v, bt in [(date,val_ms,bt_ms),(date,val_ew,bt_ew),(date,val_eq,bt_eq)]:
        bt['date'].append(d); bt['value'].append(v)

fig, ax = plt.subplots(figsize=(12, 5))
for bt, color, label in [
    (bt_ms, '#3B82F6', 'Max Sharpe (walk-forward)'),
    (bt_ew, '#F59E0B', 'Equal Weight'),
    (bt_eq, '#6B7280', '100% Equity'),
]:
    ax.plot(bt['date'], bt['value'], color=color, linewidth=2, label=label)

ax.axhline(1, color='gray', linestyle='--', linewidth=0.8)
ax.set_title(f'Backtest ({start_year}–2025): Annual Walk-Forward Rebalancing', fontsize=12, fontweight='bold')
ax.set_ylabel('Portfolio Value (₹1 invested)')
ax.legend()
fig.tight_layout()
plt.show()

for name, bt in [('Max Sharpe', bt_ms), ('Equal Weight', bt_ew), ('100% Equity', bt_eq)]:
    final = bt['value'][-1]
    n_yrs = (bt['date'][-1] - bt['date'][0]).days / 365.25
    cagr  = final ** (1 / n_yrs) - 1
    print(f"{name:25s}  Final: {final:.2f}x  CAGR: {cagr*100:.1f}%")

In [ ]:
# ── Monte Carlo block-bootstrap (5,000 paths, 10-year horizon) ────────────
BLOCK_SIZE  = 6    # 6-month blocks
N_MONTHS    = 120  # 10-year horizon
N_SIMS      = 5000
rng = np.random.default_rng(0)

ret_matrix = monthly_ret.values  # shape (T, n)
T = len(ret_matrix)

def simulate_wealth(weights, n_sims=N_SIMS, horizon=N_MONTHS, block=BLOCK_SIZE):
    paths = np.ones((n_sims, horizon + 1))
    for sim in range(n_sims):
        month = 0
        while month < horizon:
            start = rng.integers(0, T - block)
            take  = min(block, horizon - month)
            block_ret = ret_matrix[start:start + take]  # (take, n)
            for r in block_ret:
                paths[sim, month + 1] = paths[sim, month] * (1 + float(weights @ r))
                month += 1
    return paths  # (n_sims, horizon+1)

portfolios = [
    (w_ms, 'Max Sharpe',   '#3B82F6'),
    (w_mv, 'Min Variance', '#10B981'),
    (w_ew, 'Equal Weight', '#F59E0B'),
]

percentiles = [5, 25, 50, 75, 95]
months_axis = np.arange(N_MONTHS + 1)

fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharey=True)
for ax, (w, name, color) in zip(axes, portfolios):
    paths = simulate_wealth(w)
    pcts  = np.percentile(paths, percentiles, axis=0)  # (5, 121)

    ax.fill_between(months_axis, pcts[0], pcts[4], color=color, alpha=0.15, label='5–95th pct')
    ax.fill_between(months_axis, pcts[1], pcts[3], color=color, alpha=0.35, label='25–75th pct')
    ax.plot(months_axis, pcts[2], color=color, linewidth=2.2, label='Median (p50)')
    ax.axhline(1, color='gray', linestyle='--', linewidth=0.8)

    ax.set_title(f'{name}\nMedian final: {pcts[2,-1]:.1f}×  p5: {pcts[0,-1]:.1f}×', fontsize=10)
    ax.set_xlabel('Month')
    ax.xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'Yr {int(x//12)}'))
    ax.tick_params(axis='x', which='major', labelrotation=30)

axes[0].set_ylabel('Wealth (₹1 invested)')
axes[0].legend(fontsize=8)
fig.suptitle(f'Monte Carlo Wealth Paths — 10-Year Horizon ({N_SIMS:,} paths, block bootstrap)',
             fontsize=12, fontweight='bold')
fig.tight_layout()
plt.show()